In [74]:
import io
import torch
import numpy as np
import pandas as pd
import sentencepiece as spm
from gensim.models import Word2Vec
from torch.utils.data import Dataset, DataLoade
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'DataLoade' from 'torch.utils.data' (/opt/homebrew/lib/python3.11/site-packages/torch/utils/data/__init__.py)

In [ ]:
def dataframe_sentence_generator(df):
    for sentence in df:
        yield sentence

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Dataset1000WithScore.txt'
df = pd.read_csv(file_path, sep=';')

# Split the data into training and test sets
train_texts, test_texts, train_score, test_score = train_test_split(
    df['title'], 
    df['score'], 
    test_size=0.2, 
    random_state=42
)

In [ ]:
spm.SentencePieceTrainer.train(
    sentence_iterator=dataframe_sentence_generator(train_texts), 
    model_prefix='spm_Alex_week2', 
    vocab_size=2400
)

sp = spm.SentencePieceProcessor()
sp.load('spm_Alex_week2.model')

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: spm_Alex_week2
  model_type: UNIGRAM
  vocab_size: 2400
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_

True

In [ ]:
sp.encode_as_pieces('This is a test....???')

['▁Thi', 's', '▁', 'is', '▁a', '▁test', '...', '.', '?', '?', '?']

In [ ]:
tokenized_titles = [sp.encode_as_pieces(title) for title in train_texts]

In [ ]:
w2v_model = Word2Vec(
    sentences=tokenized_titles, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=4
)

In [ ]:
def title_to_embedding(title):
    tokens = sp.encode_as_pieces(title)
    embeddings = [w2v_model.wv[token] for token in tokens if token in w2v_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(100)